## Defining a KDE Function to Evaluate Six Dimensional Position-Velocity Space

**What is a KDE?**

In a kernel density estimation, each point of data is transformed into individual kernels which combine together to give a smooth probability density function for the input parameter(s). This proves useful as a regular histogram-style density estimate would not be differentiable but a kernel density estimate is.

**Using `sklearn.neighbors.KernelDensity`:**

There are six kernels that are currently avaliable with this module (gaussian, tophat, epanechnikov, exponential, linear, cosine). The methods from this module that were used are as follows:

`fit` takes in an NxM matrix of N data points and M parameters and fits them to the specified kernel and bandwidth.

`score_samples` takes in an array of points that are being queried and applies these points to the previously `fit` data. The input would be a QxM matrix of Q sets of points and M parameters and the output gives a 1xQ array of logarithmic probabilities at each of the points.

For more practical purposes the function converts the logarithmic probabilities to standard by taking an exponential. For the complete original documentation:

http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html#sklearn.neighbors.KernelDensity

**Optimizing the bandwidth and `scipy.stats.iqr`:**

The bandwidth is optimized in terms of Scott's Rule of Thumb, which follows the model: 

$bw = {1.059}{(A)}{(N)}^{(-1/5)}$, where $A = min(std(X),\frac{IQR}{1.34})$

In this case X is the inputs (an NxM matrix of N points and M parameters) and the IQR is the difference between the 75th and 25th percentile of the data. It is a measure of the dispersion similar to standard deviation or variance, but is much more robust against outliers.

To compute the IQR, we have used `scipy.stats.iqr` which takes inputs N and X, with N being the number of objects and X similarly defined as above. This outputs the IQR of our input data which is then used to compute the optimized bandwidth based on Scott's Rule of Thumb as defined above.

**Regarding inputs and `np.std` scaling:**

Each parameter space has its standard deviation taken and both `inputs` and `samples` are divided by these 6 values. This process normalizes the vecloities and positions to similar magnitudes in 6 dimensional space. It is required as the velocities magnitudes are generally much larger and span a larger range than the positions as well as differing units and because the kernel that is selected is applied analogously to all 6 dimensions.

In [10]:
#Importing the required modules
import numpy as np
from sklearn.neighbors import KernelDensity
from scipy.stats import iqr

#Defining a KDE function to quickly compute probabilities for the data set
def generate_KDE(inputs, ker):
    """
    NAME:
        generate_KDE
    
    PURPOSE:
        Given an NxM matrix for inputs and one of six avaliable ker strings, 
        outputs a function `input_DKE` that treats the density estimate as a 
        black box function that can be sampled.
    
    INPUT:
        inputs (ndarray) = An NxM matrix where N is the number of data 
                           points and M is the number of parameters.
        ker (string) = One of the 6 avaliable kernel types (gaussian, 
                       tophat, epanechnikov, exponential, linear, cosine).
    
    OUTPUT:
        input_KDE (function) = A blackbox function for the density estimate
                               used for sampling data.
                               
    HISTORY:
        2018-06-25 - Updated - Ayush Pandhi
    """
    #Scaling velocities with standard deviation
    inputs_std = np.nanstd(inputs, axis=0)
    inputs = inputs/inputs_std
    
    #Optimizing bandwidth in terms of Scott's Rule of Thumb
    shape_string = str(inputs.shape)
    objects, parameters = shape_string.split(', ')
    N_string = objects[1:]
    N = int(N_string)
    IQR = iqr(inputs)
    A = min(np.std(inputs), IQR/1.34)
    bw = 1.059 * A * N ** (-1/5.)
    
    #Fit data points to selected kernel and bandwidth
    kde = KernelDensity(kernel=ker, bandwidth=bw).fit(inputs)  
    
    def input_KDE(samples):
        """
        NAME:
            input_KDE
    
        PURPOSE:
            Given a QxM matrix for samples, evaluates the blackbox density
            estimate function at those points to output a 1xQ array of 
            density values.
    
        INPUT:
            samples (ndarray) = A QxM matrix where Q is the number of points 
                                at which the kde is being evaluated and M is 
                                the number of parameters.
                                
        OUTPUT:
            dens (ndarray) = A 1xQ array of density values for Q data points.
                               
        HISTORY:
            2018-06-25 - Updated - Ayush Pandhi
        """
        #To correct the type of information from other functions into acceptable input
        samples = np.array([samples])
        
        #Scaling samples with standard deviation
        samples = samples/inputs_std
        
        #Get the log density for selected samples and apply exponential to get normal probabilities
        log_dens = kde.score_samples(samples)
        dens = np.exp(log_dens)
        
        #Return a 1xQ array of normal probabilities for the selected sample
        return dens
    
    #Return a black box function for sampling
    return input_KDE

The following section tests this function on some mock data. For more details about the setup for the mock data, see `sampling_R^6_to_R^6.ipynb` by Michael Poon. It is serves as a good example to see the shapes and types of inputs to avoid getting dimensional errors with the function.

In [2]:
#Testing with mock data
import random

mock_data3 = []
for i in range(10):
    select_random = np.linspace(1.0, 10.0, 100)
    x4 = random.choice(select_random)
    x5 = random.choice(select_random)
    x6 = random.choice(select_random)
    x1 = random.choice(select_random)
    x2 = random.choice(select_random)
    x3 = random.choice(select_random)
    mock_data3.append([x1, x2, x3, x4, x5, x6])
print(mock_data3)

[[9.272727272727273, 2.8181818181818183, 6.090909090909091, 5.2727272727272725, 1.2727272727272727, 7.090909090909091], [7.909090909090909, 5.818181818181818, 7.7272727272727275, 4.909090909090909, 3.5454545454545454, 5.090909090909091], [8.727272727272727, 6.181818181818182, 6.181818181818182, 5.090909090909091, 9.181818181818182, 3.5454545454545454], [1.0, 8.09090909090909, 5.2727272727272725, 1.1818181818181819, 4.818181818181818, 2.6363636363636367], [5.2727272727272725, 5.090909090909091, 3.1818181818181817, 2.090909090909091, 4.272727272727273, 4.454545454545455], [5.454545454545455, 3.909090909090909, 1.0, 7.818181818181818, 9.818181818181818, 7.181818181818182], [5.818181818181818, 6.2727272727272725, 9.90909090909091, 2.1818181818181817, 8.363636363636363, 7.7272727272727275], [6.181818181818182, 2.909090909090909, 2.3636363636363638, 1.0909090909090908, 1.0909090909090908, 4.545454545454545], [2.1818181818181817, 4.363636363636363, 2.1818181818181817, 4.090909090909091, 5.181

In [11]:
a = np.array(mock_data3)
samples_random = [[1, 2, 3, 4, 5, 6]]
b = np.array(samples_random)
samples_all = [[9.272727272727273, 2.8181818181818183, 6.090909090909091, 5.2727272727272725, 1.2727272727272727, 7.090909090909091], [7.909090909090909, 5.818181818181818, 7.7272727272727275, 4.909090909090909, 3.5454545454545454, 5.090909090909091], [8.727272727272727, 6.181818181818182, 6.181818181818182, 5.090909090909091, 9.181818181818182, 3.5454545454545454], [1.0, 8.09090909090909, 5.2727272727272725, 1.1818181818181819, 4.818181818181818, 2.6363636363636367], [5.2727272727272725, 5.090909090909091, 3.1818181818181817, 2.090909090909091, 4.272727272727273, 4.454545454545455], [5.454545454545455, 3.909090909090909, 1.0, 7.818181818181818, 9.818181818181818, 7.181818181818182], [5.818181818181818, 6.2727272727272725, 9.90909090909091, 2.1818181818181817, 8.363636363636363, 7.7272727272727275], [6.181818181818182, 2.909090909090909, 2.3636363636363638, 1.0909090909090908, 1.0909090909090908, 4.545454545454545], [2.1818181818181817, 4.363636363636363, 2.1818181818181817, 4.090909090909091, 5.181818181818182, 5.545454545454546], [1.0, 10.0, 3.909090909090909, 5.545454545454546, 7.181818181818182, 2.909090909090909]]
c = np.array(samples_all)
c

array([[ 9.27272727,  2.81818182,  6.09090909,  5.27272727,  1.27272727,
         7.09090909],
       [ 7.90909091,  5.81818182,  7.72727273,  4.90909091,  3.54545455,
         5.09090909],
       [ 8.72727273,  6.18181818,  6.18181818,  5.09090909,  9.18181818,
         3.54545455],
       [ 1.        ,  8.09090909,  5.27272727,  1.18181818,  4.81818182,
         2.63636364],
       [ 5.27272727,  5.09090909,  3.18181818,  2.09090909,  4.27272727,
         4.45454545],
       [ 5.45454545,  3.90909091,  1.        ,  7.81818182,  9.81818182,
         7.18181818],
       [ 5.81818182,  6.27272727,  9.90909091,  2.18181818,  8.36363636,
         7.72727273],
       [ 6.18181818,  2.90909091,  2.36363636,  1.09090909,  1.09090909,
         4.54545455],
       [ 2.18181818,  4.36363636,  2.18181818,  4.09090909,  5.18181818,
         5.54545455],
       [ 1.        , 10.        ,  3.90909091,  5.54545455,  7.18181818,
         2.90909091]])

In [12]:
blackbox = generate_KDE(a, 'epanechnikov')
blackbox(c)

[2.86542513 2.14476106 2.62889223 2.09556793 2.93983752 1.71469102]
[[3.23607383 1.31398405 2.31691091 2.51613283 0.43292436 4.13538592]
 [2.76018062 2.71274126 2.93936458 2.34260643 1.20600357 2.96899502]
 [3.04571655 2.88228759 2.35149167 2.42936963 3.12324002 2.06769296]
 [0.34898835 3.77240582 2.00568407 0.56396081 1.63892793 1.53751528]
 [1.84012041 2.3736486  1.21032659 0.99777681 1.45338892 2.59787064]
 [1.90357284 1.82262303 0.38038836 3.73081765 3.3397022  4.18840369]
 [2.0304777  2.92467417 3.76930282 1.04115841 2.8449315  4.50651029]
 [2.15738256 1.35637063 0.89909975 0.52057921 0.37107802 2.65088841]
 [0.76142914 2.03455595 0.82993824 1.95217202 1.7626206  3.23408386]
 [0.34898835 4.66252404 1.48697267 2.64627763 2.44293031 1.69656858]]
0.7322594897024284
[2.86542513 2.14476106 2.62889223 2.09556793 2.93983752 1.71469102]
[[3.23607383 1.31398405 2.31691091 2.51613283 0.43292436 4.13538592]
 [2.76018062 2.71274126 2.93936458 2.34260643 1.20600357 2.96899502]
 [3.04571655 2.8

array([0.50207844, 0.50207844, 0.50207844, 0.50207844, 0.50207844,
       0.50207844, 0.50207844, 0.50207844, 0.50207844, 0.50207844])